In [83]:
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score 
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [84]:
def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

In [126]:
parent_path = '/home/tushar/Downloads/dataframe_'#'/home/tushar/accuracy_final_new_'#'/home/tushar/Downloads/SVM_accuracy_final_new_'#

In [127]:
pred_col = 'pred'
actual_col = 'actual'

In [131]:
accuracy_df = pd.DataFrame()
for division in [1,2,3,4,5]:
    accuracy_df = accuracy_df.append(pd.read_csv(parent_path+str(division)+'.csv'))
accuracy_df = accuracy_df.rename(columns={'predicted_lable':'pred','true_label':'actual','words':'word'})


In [132]:
print(len(accuracy_df[accuracy_df[pred_col]==accuracy_df[actual_col]])/len(accuracy_df))

0.9631645757118548


In [133]:
unique_tags = accuracy_df[actual_col].unique()

In [134]:
unique_tags

array(['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT',
       'PRON', 'NUM', 'X'], dtype=object)

In [135]:
per_pos_df = pd.DataFrame()
for tag in unique_tags:
    temp_accuracy_df = accuracy_df[accuracy_df[actual_col]==tag]
    accuracy = len(temp_accuracy_df[temp_accuracy_df[pred_col]==temp_accuracy_df[actual_col]])/len(temp_accuracy_df)
    print(tag,len(temp_accuracy_df),accuracy)
    row = {}
    row['tag'] = tag
    row['count'] = len(temp_accuracy_df)
    row['accuracy'] = accuracy
    per_pos_df = per_pos_df.append(row,ignore_index=True)

DET 137019 0.9933220940161583
NOUN 275558 0.9585894802546107
ADJ 83721 0.8876625936145053
VERB 182750 0.9610396716826265
ADP 144766 0.9800091181631045
. 147565 0.999898349879714
ADV 56239 0.9123028503351767
CONJ 38151 0.9960682550916097
PRT 29829 0.9175969693922023
PRON 49334 0.9829123930757693
NUM 14874 0.8967997848594863
X 1386 0.20995670995670995


In [137]:
confusion_matrix = accuracy_df.groupby([pred_col,actual_col])['word'].count().unstack().fillna(0)

In [138]:
print(per_pos_df.round(2).to_latex())  

\begin{tabular}{lrrl}
\toprule
{} &  accuracy &     count &   tag \\
\midrule
0  &      0.99 &  137019.0 &   DET \\
1  &      0.96 &  275558.0 &  NOUN \\
2  &      0.89 &   83721.0 &   ADJ \\
3  &      0.96 &  182750.0 &  VERB \\
4  &      0.98 &  144766.0 &   ADP \\
5  &      1.00 &  147565.0 &     . \\
6  &      0.91 &   56239.0 &   ADV \\
7  &      1.00 &   38151.0 &  CONJ \\
8  &      0.92 &   29829.0 &   PRT \\
9  &      0.98 &   49334.0 &  PRON \\
10 &      0.90 &   14874.0 &   NUM \\
11 &      0.21 &    1386.0 &     X \\
\bottomrule
\end{tabular}



In [139]:
confusion_matrix = confusion_matrix.iloc[1:]

In [140]:
i = 0
prec_recall = pd.DataFrame()
for idx in confusion_matrix.index:
    row = {}
    row['tag'] = idx
    row['recall'] = rec = recall(i,np.array(confusion_matrix).T)
    row['precision'] = prec = precision(i,np.array(confusion_matrix).T)
    row['F1 score'] = 2*(rec * prec) / (rec + prec)


    i+=1
    prec_recall = prec_recall.append(row,ignore_index=True)
    

In [141]:
print(confusion_matrix.astype(int).to_latex(index=True))  

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
actual &       . &    ADJ &     ADP &    ADV &   CONJ &     DET &    NOUN &    NUM &   PRON &    PRT &    VERB &    X \\
pred &         &        &         &        &        &         &         &        &        &        &         &      \\
\midrule
.    &  147550 &      1 &      22 &      0 &      0 &       2 &       6 &      1 &      0 &      0 &       0 &   24 \\
ADJ  &       3 &  74307 &      48 &   1801 &      3 &       7 &    4375 &    136 &      3 &     48 &     811 &   64 \\
ADP  &       4 &     89 &  141872 &   1155 &     31 &     482 &     123 &     14 &    266 &   1775 &     135 &   29 \\
ADV  &       0 &   1557 &     708 &  51307 &     46 &      93 &     225 &      1 &      3 &    241 &     139 &    6 \\
CONJ &       1 &      1 &     140 &    134 &  38001 &      34 &       5 &      0 &      0 &      0 &       1 &    4 \\
DET  &       1 &      2 &     341 &    177 &     57 &  136104 &     116 &      0 &    504 &      5 &       5 &   19 

In [142]:
print(prec_recall.round(2).to_latex(index=True))  

\begin{tabular}{lrrrl}
\toprule
{} &  F1 score &  precision &  recall &   tag \\
\midrule
0  &      1.00 &       1.00 &    1.00 &     . \\
1  &      0.90 &       0.91 &    0.89 &   ADJ \\
2  &      0.98 &       0.97 &    0.98 &   ADP \\
3  &      0.93 &       0.94 &    0.91 &   ADV \\
4  &      0.99 &       0.99 &    1.00 &  CONJ \\
5  &      0.99 &       0.99 &    0.99 &   DET \\
6  &      0.95 &       0.94 &    0.96 &  NOUN \\
7  &      0.94 &       0.98 &    0.90 &   NUM \\
8  &      0.99 &       0.99 &    0.98 &  PRON \\
9  &      0.93 &       0.94 &    0.92 &   PRT \\
10 &      0.96 &       0.96 &    0.96 &  VERB \\
11 &      0.32 &       0.63 &    0.21 &     X \\
\bottomrule
\end{tabular}



In [143]:
prec_recall.mean()

F1 score     0.905478
precision    0.937532
recall       0.891454
dtype: float64

In [39]:
np.array(confusion_matrix).T

array([[1.47526e+05, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 4.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00],
       [1.37000e+02, 2.03500e+03, 2.28400e+03, 7.20000e+02, 0.00000e+00,
        1.86200e+03, 7.38980e+04, 0.00000e+00, 5.60000e+01, 0.00000e+00,
        2.70300e+03],
       [3.39100e+03, 0.00000e+00, 1.21986e+05, 3.68000e+02, 5.25000e+02,
        5.67900e+03, 7.95700e+03, 0.00000e+00, 1.08000e+03, 1.91400e+03,
        1.82500e+03],
       [3.56400e+03, 1.75500e+03, 5.40100e+03, 2.37100e+03, 1.10000e+01,
        1.16100e+03, 2.78750e+04, 0.00000e+00, 6.02400e+03, 0.00000e+00,
        8.07000e+03],
       [1.12000e+03, 0.00000e+00, 3.48900e+03, 0.00000e+00, 3.06080e+04,
        1.20000e+01, 4.49000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        2.46100e+03],
       [1.13100e+03, 0.00000e+00, 4.94000e+02, 0.00000e+00, 0.00000e+00,
        1.31896e+05, 2.39900e+03, 0.00000e+00, 5.77000e+02, 0.00000e+00,
        4.7800

In [82]:
accuracy_df

,pred,actual,word
0,DET,DET,The
1,NOUN,NOUN,quarrel
2,VERB,VERB,ended
3,ADP,ADP,in
4,DET,DET,a
...,...,...,...
248619,VERB,VERB,gather
248620,ADP,DET,that
248621,DET,ADJ,post-attack
248622,NOUN,NOUN,reconnaissance
